# MoI ML CNN Experiment

- Train op fashion mnist
- Laat experimenten zien: kies 2 - 3 params, kies een range en voer tests uit. Doel is 93% accuracy.

In [6]:
import torch
import torch.nn as nn
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../")

In [9]:
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
import gin


In [10]:
# Load config for this notebook
gin.parse_config_file("cnn.gin")

ParsedConfigFileIncludesAndImports(filename='cnn.gin', imports=[], includes=[])

In [ ]:
# Get MNIST data
train_dataloader, test_dataloader = make_dataset.get_MNIST()